In [1]:
import setup  # notebooks/setup.py
setup.init()

In [2]:
from mypermit import permit_client

permit = permit_client

In [5]:
from django.contrib.auth import get_user_model
from documents.models import Document


User = get_user_model()

admin_users = User.objects.filter(is_superuser=True)
documents = Document.objects.all()

In [6]:
from permit import PermitApiError

async def get_or_create_resource_instance(django_instance, resource_key="document"):
    _key = f"{resource_key}:{django_instance.id}"
    try:
        _resource = await permit.api.resource_instances.get(_key)
    except PermitApiError as e:
        if e.status_code == 404:
            payload = {
                "key": f"{django_instance.id}",
                "resource": resource_key,
                "tenant": "default",
                
            }
            _resource = await permit.api.resource_instances.create(payload)
        else:
            raise e
    return _resource

In [7]:
async def assign_user_ownership(django_user, django_instance, resource_key="document"):
    _key = f"{resource_key}:{django_instance.id}"
    return await permit.api.users.assign_role({
        "user": f"{django_user.id}",
        "role": "admin",
        "resource_instance": _key,
        "tenant": "default",
    })

In [8]:
for obj in documents:
    response = await get_or_create_resource_instance(obj, resource_key="document")
    role = await assign_user_ownership(obj.owner, obj, resource_key="document")
    print(response)
    

key='6' tenant='default' resource='document' id=UUID('7f4a32ee-5bce-4784-884e-ccc0fb1dd9e4') organization_id=UUID('c5928800-1737-4973-a296-ec1c24fa0a1a') project_id=UUID('f2327095-7e49-425e-8726-87b2a85e7ee6') environment_id=UUID('a3856ddc-91af-43ac-8b22-08200a8855ed') created_at=datetime.datetime(2025, 6, 4, 18, 28, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 6, 4, 18, 28, tzinfo=datetime.timezone.utc) resource_id=UUID('3f71f9fd-14f4-4702-88f2-6745a0b68003') tenant_id=UUID('f99e0569-2824-4fe5-a820-43e804eb5d47') attributes=None relationships=None
key='11' tenant='default' resource='document' id=UUID('386858b2-6f87-44eb-9903-a488fc5ec534') organization_id=UUID('c5928800-1737-4973-a296-ec1c24fa0a1a') project_id=UUID('f2327095-7e49-425e-8726-87b2a85e7ee6') environment_id=UUID('a3856ddc-91af-43ac-8b22-08200a8855ed') created_at=datetime.datetime(2025, 6, 4, 18, 28, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 6, 4, 18, 28, tzinfo=datetime.timezone.u

In [9]:
for admin_u in admin_users:
    for obj in documents:
        response = await get_or_create_resource_instance(obj, resource_key="document")
        role = await assign_user_ownership(admin_u, obj, resource_key="document")
        print(response)

key='6' tenant='default' resource='document' id=UUID('7f4a32ee-5bce-4784-884e-ccc0fb1dd9e4') organization_id=UUID('c5928800-1737-4973-a296-ec1c24fa0a1a') project_id=UUID('f2327095-7e49-425e-8726-87b2a85e7ee6') environment_id=UUID('a3856ddc-91af-43ac-8b22-08200a8855ed') created_at=datetime.datetime(2025, 6, 4, 18, 28, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 6, 4, 18, 28, tzinfo=datetime.timezone.utc) resource_id=UUID('3f71f9fd-14f4-4702-88f2-6745a0b68003') tenant_id=UUID('f99e0569-2824-4fe5-a820-43e804eb5d47') attributes=None relationships=None
key='11' tenant='default' resource='document' id=UUID('386858b2-6f87-44eb-9903-a488fc5ec534') organization_id=UUID('c5928800-1737-4973-a296-ec1c24fa0a1a') project_id=UUID('f2327095-7e49-425e-8726-87b2a85e7ee6') environment_id=UUID('a3856ddc-91af-43ac-8b22-08200a8855ed') created_at=datetime.datetime(2025, 6, 4, 18, 28, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 6, 4, 18, 28, tzinfo=datetime.timezone.u

In [ ]:
random_user = User.objects.all().order_by("?")[0]
random_doc = Document.objects.all().order_by("?")[0]

await permit.check(f"{random_user.id}", "update", f"document:{random_doc.id}")

False